Kaart NL met urbanisatiegraad per gemeente.

In [76]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
gemeenten = gpd.read_file('WijkBuurtkaart_2020_v3\gemeente_2020_v3.shp')

if gemeenten.crs is None:
    gemeenten = gemeenten.set_crs(epsg=28992)  # Set it to EPSG:4326 or the appropriate one
gemeenten = gemeenten.to_crs(epsg=28992)

adressen_raw = pd.read_csv('84799NED_TypedDataSet_07102024_140353.csv', delimiter=';', encoding='utf-8')

adressen_raw.columns = adressen_raw.columns.str.strip()
adressen_raw['SoortRegio_2'] = adressen_raw['SoortRegio_2'].str.strip()

# Check unique values in 'SoortRegio_2'
print(adressen_raw['SoortRegio_2'].unique())

# Filter and select relevant columns
adressen = adressen_raw[adressen_raw['SoortRegio_2'] == 'Gemeente'][['Gemeentenaam_1', 'Omgevingsadressendichtheid_117']]

# Display the filtered dataframe
#print(adressen)

#print(gemeenten)

print("Gemeenten Columns:", gemeenten.columns.tolist())

In [ ]:
# Inspect columns in 'adressen'
print("Adressen Columns:", adressen.columns.tolist())

# Inspect columns in 'gemeenten'
print("Gemeenten Columns:", gemeenten.columns.tolist())

# Identify overlapping columns
overlapping_columns = set(adressen.columns).intersection(set(gemeenten.columns))
print("Overlapping Columns:", overlapping_columns)


In [ ]:
"""
# Rename the column in the address data (if needed) to match the shapefile's column
adressen = adressen.rename(columns={'Gemeentenaam_1': 'gemeente'})

print("Adressen Columns:", adressen.columns)

# Merge the GeoDataFrame with the DataFrame on the common column with unique suffixes
gemeenten = gemeenten.merge(
    adressen,
    left_on='GM_NAAM',
    right_on='gemeente',
)

# Check the merged GeoDataFrame
print(gemeenten.head())
"""
print(gemeenten.STED)
# Filter rows where 'STED_numeric' is between 0 and 5 (inclusive)
gemeenten_filtered = gemeenten[
    (gemeenten['STED'] >= 0) &
    (gemeenten['STED'] <= 5)
].copy()

print(gemeenten_filtered.STED)
# Alternatively, using the 'between' method
# gemeenten_filtered = gemeenten[gemeenten['STED_numeric'].between(0, 5, inclusive='both')].copy()


#'Shape_Leng', 'Shape_Area', 'geometry'

In [ ]:
import geopandas as gpd
import plotly.express as px
import json

# Step 2: Convert GeoDataFrame to GeoJSON
gemeenten_geojson = json.loads(gemeenten_filtered.to_json())

# Step 3: Assign 'GM_CODE' as 'id' for each GeoJSON feature
for feature in gemeenten_geojson['features']:
    feature['id'] = feature['properties']['GM_CODE']

# Step 4: Create the choropleth map with square dimensions
fig = px.choropleth_mapbox(
    gemeenten_filtered,                  # Your GeoDataFrame
    geojson=gemeenten_geojson,           # The GeoJSON data
    locations='GM_CODE',                 # Column in DataFrame matching GeoJSON 'id'
    color='STED',                        # Column to determine color intensity
    hover_name='GM_NAAM',                # Column to display on hover
    hover_data={'STED': True, 'GM_CODE': False},  # Data to show/hide on hover
    color_continuous_scale=px.colors.sequential.OrRd[::-1],  # Inverted 'OrRd' colormap
    mapbox_style='carto-positron',        # Base map style
    zoom=6,                               # Initial zoom level
    center={"lat": 52.1326, "lon": 5.2913},  # Center of the Netherlands
    opacity=0.8,                          # Transparency of the polygons
    labels={'STED': 'Urbanisation level'},   # Legend label
    height=800,                           # Set the height of the map
    width=800                             # Set the width of the map to make it square
)

# Step 5: Define integer tick values based on 'STED' range
min_sted = int(gemeenten_filtered['STED'].min())
max_sted = int(gemeenten_filtered['STED'].max())
tick_values = list(range(min_sted, max_sted + 1, 1))  # Integer ticks

# Step 6: Update the color bar to show only integer ticks
fig.update_coloraxes(
    colorbar=dict(
        title="Urbanisation level",
        tickmode='array',
        tickvals=tick_values,
        ticktext=[str(tick) for tick in tick_values],
    )
)

# Step 7: Add a centered title and adjust layout
fig.update_layout(
    title_text='Address Density per Municipality in 2020',
    title_x=0.5,  # Center the title
    margin={"r":0,"t":50,"l":0,"b":0},  # Remove unnecessary margins
)

# Step 8: Display the interactive map
fig.show()


In [ ]:
""" wijken = gpd.read_file('WijkBuurtkaart_2020_v3\wijk_2020_v3.shp')

if wijken.crs is None:
    wijken = wijken.set_crs(epsg=28992)  # Set it to EPSG:4326 or the appropriate one
wijken = wijken.to_crs(epsg=28992)

print("Gemeenten Columns:", wijken.columns.tolist())

print(wijken.STED)
# Filter rows where 'STED_numeric' is between 0 and 5 (inclusive)
wijken_filtered = wijken[
    (wijken['STED'] >= 0) &
    (wijken['STED'] <= 5)
].copy()

print(wijken_filtered.STED) """

In [ ]:
""" 

# Step 2: Convert GeoDataFrame to GeoJSON
wijken_geojson = json.loads(wijken_filtered.to_json())

# Step 3: Assign 'GM_CODE' as 'id' for each GeoJSON feature
for feature in wijken_geojson['features']:
    feature['id'] = feature['properties']['WK_CODE']

# Step 4: Create the choropleth map with square dimensions
fig = px.choropleth_mapbox(
    wijken_filtered,                  # Your GeoDataFrame
    geojson=wijken_geojson,           # The GeoJSON data
    locations='WK_CODE',                 # Column in DataFrame matching GeoJSON 'id'
    color='STED',                        # Column to determine color intensity
    hover_name='WK_NAAM',                # Column to display on hover
    hover_data={'STED': True, 'WK_CODE': False},  # Data to show/hide on hover
    color_continuous_scale=px.colors.sequential.OrRd[::-1],  # Inverted 'OrRd' colormap
    mapbox_style='carto-positron',        # Base map style
    zoom=6,                               # Initial zoom level
    center={"lat": 52.1326, "lon": 5.2913},  # Center of the Netherlands
    opacity=0.8,                          # Transparency of the polygons
    labels={'STED': 'Urbanisation level'},   # Legend label
    height=800,                           # Set the height of the map
    width=800                             # Set the width of the map to make it square
)

# Step 5: Define integer tick values based on 'STED' range
min_sted = int(wijken_filtered['STED'].min())
max_sted = int(wijken_filtered['STED'].max())
tick_values = list(range(min_sted, max_sted + 1, 1))  # Integer ticks

# Step 6: Update the color bar to show only integer ticks
fig.update_coloraxes(
    colorbar=dict(
        title="Urbanisation level",
        tickmode='array',
        tickvals=tick_values,
        ticktext=[str(tick) for tick in tick_values],
    )
)

# Step 7: Add a centered title and adjust layout
fig.update_layout(
    title_text='Address Density per Neighbourhood in 2020',
    title_x=0.5,  # Center the title
    margin={"r":0,"t":50,"l":0,"b":0},  # Remove unnecessary margins
)

# Step 8: Display the interactive map
fig.show() """